In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('m.csv')
df.head()

,sex,suicides_no,population,suicides_pop,gdp_per_capita,lvl_gdp,GDP_for_group,Boomers,G_I_Generation,GenerationX,GenerationZ,Millenials,Silent
0,1.0,21.0,312900.0,6.71,796.0,0.0,249068400.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1.0,16.0,308000.0,5.19,796.0,0.0,245168000.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,14.0,289700.0,4.83,796.0,0.0,230601200.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1.0,1.0,21800.0,4.59,796.0,0.0,17352800.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1.0,9.0,274300.0,3.28,796.0,0.0,218342800.0,1.0,0.0,0.0,0.0,0.0,0.0


In [3]:
X = df.drop(('sex'), axis=1)
y = df['sex']
feature_names = X.columns
print (feature_names)

Index(['suicides_no', 'population', 'suicides_pop', 'gdp_per_capita',
       'lvl_gdp', 'GDP_for_group', 'Boomers', 'G_I_Generation', 'GenerationX',
       'GenerationZ', 'Millenials', 'Silent'],
      dtype='object')


In [4]:
print (X.shape)
print (y.shape)
N, d = X.shape

(27820, 12)
(27820,)


In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 17)

N_train, _ = X_train.shape 
N_test,  _ = X_test.shape 
print (N_train, N_test)

19474 8346


In [6]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

bagging = BaggingClassifier(DecisionTreeClassifier(max_depth=25, random_state=99), max_samples=1.0, max_features=1.0)
bagging.fit(X_train, y_train)
predict0 = bagging.predict(X_test)
pred_b = bagging.predict_proba(X_test)

acc_b = metrics.accuracy_score(y_test,  predict0)
print (acc_b)

0.7910376228133238


In [7]:
roc_b = metrics.roc_auc_score(y_test, predict0)
print(roc_b)

0.7908569262751675


In [8]:
from sklearn import ensemble
rf = ensemble.RandomForestClassifier(max_depth=25 ,n_estimators=100, random_state=99)
rf.fit(X_train, y_train)
predict_rf = rf.predict(X_test)

acc_r = metrics.accuracy_score(y_test,  predict_rf)
print(acc_r)

0.8006230529595015


In [9]:
roc_r = metrics.roc_auc_score(y_test, predict_rf)
print(roc_r)

0.8005337843270126


In [10]:
from sklearn.linear_model import SGDClassifier

classifier = SGDClassifier(loss='log',random_state=2, max_iter = 1000)
classifier.fit(X_train, y_train)
predict_cl = classifier.predict(X_test)
pred_cl = classifier.predict_proba(X_test)

acc_c = metrics.accuracy_score(y_test,  predict_cl)
print(acc_c)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


0.4972441888329739


In [11]:
roc_c = metrics.roc_auc_score(y_test, predict_cl)
print(roc_c)

0.5


In [12]:
from sklearn import neighbors

knn = neighbors.KNeighborsClassifier(n_neighbors=17)
knn.fit(X_train, y_train)
predict_k = knn.predict(X_test)
pred_k = knn.predict_proba(X_test)

acc_k = metrics.accuracy_score(y_test, predict_k)
print(acc_k)

0.51294033069734


In [13]:
roc_k = metrics.roc_auc_score(y_test, predict_k)
print(roc_k)

0.5130292188774163


In [14]:
per_b = (acc_b + roc_b) / 2
per_r = (acc_r + roc_r) / 2
per_c = (acc_c + roc_c) / 2
per_k = (acc_k + roc_k) / 2

In [15]:
t = [per_b, per_r, per_c, per_k]
t = max(t)

In [16]:
if per_b == t:
    print ('Bagging Classifier accuracy score:', acc_b, 'roc auc score:', roc_b)
elif per_r == t:
    print ('Random Forest Classifier accuracy score:', acc_r, 'roc auc score:', roc_r)
elif per_c == t:
    print ('Linear classifier accuracy score:', acc_c, 'roc auc score:', roc_c)
elif per_k == t:
    print ('K Neighbors Classifier accuracy score:', acc_k, 'roc auc score:', roc_k)

Random Forest Classifier accuracy score: 0.8006230529595015 roc auc score: 0.8005337843270126


In [17]:
from sklearn.ensemble import VotingClassifier

vot = VotingClassifier(estimators=[('lr', classifier), ('rf', rf), ('bg', bagging), ('kn', knn)], voting='hard')

vot.fit(X_train, y_train)
predict_v = vot.predict(X_test)

roc_v = metrics.roc_auc_score(y_test, predict_v)
print (roc_v)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


0.7993197192966336


In [18]:
acc_v = metrics.accuracy_score(y_test, predict_v)
print (acc_v)

0.7994248741912293
